<a href="https://colab.research.google.com/github/apester/IME/blob/main/Lab_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler


In [ ]:
# Generate synthetic classification dataset
X, y = make_classification(n_samples=5000, n_features=20, n_classes=2, random_state=42)
X = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Overfitting-prone model (no regularization)
model_overfit = keras.Sequential([
    layers.Dense(256, activation='relu', input_shape=(X.shape[1],)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model_overfit.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the overfitting model
history_overfit = model_overfit.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=0)

In [ ]:
# Regularized Model (L2 + Dropout)
model_regularized = keras.Sequential([
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01), input_shape=(X.shape[1],)),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.3),
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(1, activation='sigmoid')
])

model_regularized.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train with Early Stopping
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history_regularized = model_regularized.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=0, callbacks=[early_stopping])


In [ ]:
# Plot Training Loss
plt.figure(figsize=(10,5))
plt.plot(history_overfit.history['loss'], label='Overfitting Model - Training Loss')
plt.plot(history_overfit.history['val_loss'], label='Overfitting Model - Validation Loss')
plt.plot(history_regularized.history['loss'], label='Regularized Model - Training Loss', linestyle='dashed')
plt.plot(history_regularized.history['val_loss'], label='Regularized Model - Validation Loss', linestyle='dashed')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Effect of Regularization on Overfitting')
plt.show()


In [ ]:
# Plot Accuracy
plt.figure(figsize=(10,5))
plt.plot(history_overfit.history['accuracy'], label='Overfitting Model - Training Accuracy')
plt.plot(history_overfit.history['val_accuracy'], label='Overfitting Model - Validation Accuracy')
plt.plot(history_regularized.history['accuracy'], label='Regularized Model - Training Accuracy', linestyle='dashed')
plt.plot(history_regularized.history['val_accuracy'], label='Regularized Model - Validation Accuracy', linestyle='dashed')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Effect of Regularization on Model Performance')
plt.show()
